# Apply PCM to argo reference data base

First try

In [1]:
import xarray as xr
import numpy as np
import numpy.ma as ma
import pandas as pd

import pyxpcm
from pyxpcm.models import pcm

import Plotter
from Plotter import Plotter

from BIC_calculation import *

from classif_functions import *

import matplotlib as mpl
import matplotlib.pyplot as plt

import cartopy.feature as cfeature
import cartopy.crs as ccrs

import argopy
argopy.set_options(src='localftp', local_ftp='/home/coriolis_exp/spool/co05/co0508/')
from argopy import DataFetcher as ArgoDataFetcher
argo_loader = ArgoDataFetcher()

### __User inputs__

Reference data selection

In [2]:
# geographical extent
geo_extent = [-70, -43, 32, 49] # [min lon, max lon, min lat, max lat]
# depth for interpolation
max_depth = 1000
# chose season ('DJF', 'MAM', 'JJA', 'SON' or 'all')
season = ['all']

Float you want to correct

In [3]:
float_WMO = 4900136

Model parameters

In [4]:
K=4 # number of classes

## 1. Load argo reference database

Read wmo boxes latlon: load txt file

In [5]:
ds = get_refdata(geo_extent=geo_extent, 
                 WMOboxes_latlon='WMO_boxes_latlon.txt', 
                 wmo_boxes='wmo_boxes_argo.mat', 
                 ref_path = '/home1/homedir5/perso/agarciaj/EARISE/OW/matlabow/data/climatology/',
                 season = season)

/home1/homedir5/perso/agarciaj/EARISE/OW/matlabow/data/climatology/argo_7307.mat
/home1/homedir5/perso/agarciaj/EARISE/OW/matlabow/data/climatology/argo_7406.mat
/home1/homedir5/perso/agarciaj/EARISE/OW/matlabow/data/climatology/argo_7306.mat
/home1/homedir5/perso/agarciaj/EARISE/OW/matlabow/data/climatology/argo_7405.mat
/home1/homedir5/perso/agarciaj/EARISE/OW/matlabow/data/climatology/argo_7305.mat


In [10]:
ds['n_profiles'] = ds.n_profiles.values

In [11]:
ds

<xarray.Dataset>
Dimensions:     (n_pres: 1049, n_profiles: 4493)
Coordinates:
    long        (n_profiles) float64 -60.75 -60.63 -65.71 ... -55.99 -56.38
    lat         (n_profiles) float64 40.49 40.36 40.07 ... 35.9 35.15 34.61
    dates       (n_profiles) datetime64[ns] 2013-10-10T11:26:16 ... 2016-09-06T23:52:00
  * n_profiles  (n_profiles) int64 0 1 2 3 4 5 ... 4487 4488 4489 4490 4491 4492
Dimensions without coordinates: n_pres
Data variables:
    pres        (n_pres, n_profiles) float64 1.0 1.0 5.4 5.3 ... nan nan nan nan
    temp        (n_pres, n_profiles) float64 23.43 21.57 26.01 ... nan nan nan
    sal         (n_pres, n_profiles) float64 35.65 35.87 36.15 ... nan nan nan
    source      (n_profiles) object ['1901534_081.BRECKOWENS'] ... ['4901755_120.BlairGreenan']
Attributes:
    __header__:   b'MATLAB 5.0 MAT-file, Platform: GLNXA64, Created on: Tue N...
    __version__:  1.0
    __globals__:  1.0

__load float data__

In [7]:
ds_f = argo_loader.float([float_WMO]).to_xarray()
ds_f = ds_f.argo.point2profile()
print(ds_f)

<xarray.Dataset>
Dimensions:                (N_LEVELS: 72, N_PROF: 119)
Coordinates:
  * N_LEVELS               (N_LEVELS) int64 0 1 2 3 4 5 6 ... 66 67 68 69 70 71
  * N_PROF                 (N_PROF) int64 0 1 2 3 4 5 ... 114 115 116 117 118
    LONGITUDE              (N_PROF) float64 -55.45 -55.74 ... -51.06 -49.78
    TIME                   (N_PROF) datetime64[ns] 2002-07-06T07:48:00 ... 2005-09-28T07:40:00
    LATITUDE               (N_PROF) float64 44.46 44.53 44.61 ... 38.8 38.67
Data variables:
    CONFIG_MISSION_NUMBER  (N_PROF) int64 0 0 0 0 0 0 0 0 0 ... 0 0 0 0 0 0 0 0
    CYCLE_NUMBER           (N_PROF) int64 1 2 3 4 5 6 ... 115 116 117 118 119
    DATA_MODE              (N_PROF) <U1 'D' 'D' 'D' 'D' 'D' ... 'D' 'D' 'D' 'D'
    DIRECTION              (N_PROF) <U1 'A' 'A' 'A' 'A' 'A' ... 'A' 'A' 'A' 'A'
    PLATFORM_NUMBER        (N_PROF) int64 4900136 4900136 ... 4900136 4900136
    POSITION_QC            (N_PROF) int64 1 1 1 1 1 1 1 1 1 ... 1 1 1 1 1 1 1 1
    PRES         

delate float profiles in reference dataset

In [8]:
nan_matrix = np.empty((len(ds_f['N_PROF'].values), len(ds['n_pres'].values) - len(ds_f['N_LEVELS'].values)))
nan_matrix.fill(np.nan)
ds_fc = xr.Dataset(
         data_vars=dict(
             pres=(["n_profiles", "n_pres"], np.concatenate((ds_f['PRES'].values, nan_matrix),axis=1)),
             temp=(["n_profiles", "n_pres"], np.concatenate((ds_f['TEMP'].values, nan_matrix),axis=1)),
             sal=(["n_profiles", "n_pres"], np.concatenate((ds_f['TEMP'].values, nan_matrix),axis=1)),
             source=(["n_profiles"], nan_matrix[:,1]),
         ),
         coords=dict(
             long=(["n_profiles"], ds_f['LONGITUDE'].values),
             lat=(["n_profiles"], ds_f['LATITUDE'].values),
             dates=(["n_profiles"], ds_f['TIME'].values),
         )
     )
ds_fc['n_profiles'] = ds_fc.n_profiles.values + len(ds.n_profiles.values)
ds_fc

<xarray.Dataset>
Dimensions:     (n_pres: 1049, n_profiles: 119)
Coordinates:
    long        (n_profiles) float64 -55.45 -55.74 -55.91 ... -51.06 -49.78
    lat         (n_profiles) float64 44.46 44.53 44.61 ... 38.12 38.8 38.67
    dates       (n_profiles) datetime64[ns] 2002-07-06T07:48:00 ... 2005-09-28T07:40:00
  * n_profiles  (n_profiles) int64 4493 4494 4495 4496 ... 4608 4609 4610 4611
Dimensions without coordinates: n_pres
Data variables:
    pres        (n_profiles, n_pres) float64 7.3 9.5 19.3 29.8 ... nan nan nan
    temp        (n_profiles, n_pres) float64 12.23 12.23 11.0 ... nan nan nan
    sal         (n_profiles, n_pres) float64 12.23 12.23 11.0 ... nan nan nan
    source      (n_profiles) float64 nan nan nan nan nan ... nan nan nan nan nan

In [12]:
xr.combine_by_coords([ds, ds_fc])

<xarray.Dataset>
Dimensions:     (n_pres: 1049, n_profiles: 4612)
Coordinates:
    long        (n_profiles) float64 -60.75 -60.63 -65.71 ... -51.06 -49.78
    lat         (n_profiles) float64 40.49 40.36 40.07 ... 38.12 38.8 38.67
    dates       (n_profiles) datetime64[ns] 2013-10-10T11:26:16 ... 2005-09-28T07:40:00
  * n_profiles  (n_profiles) int64 0 1 2 3 4 5 ... 4606 4607 4608 4609 4610 4611
Dimensions without coordinates: n_pres
Data variables:
    pres        (n_pres, n_profiles) float64 1.0 1.0 5.4 5.3 ... nan nan nan nan
    temp        (n_pres, n_profiles) float64 23.43 21.57 26.01 ... nan nan nan
    sal         (n_pres, n_profiles) float64 35.65 35.87 36.15 ... nan nan nan
    source      (n_profiles) object ['1901534_081.BRECKOWENS'] ... nan

plot dataset

In [ ]:
fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(6, 4), dpi=120, facecolor='w', edgecolor='k')
sc = ax.pcolor(np.tile(ds['n_profiles'], (len(ds['n_pres']),1)), ds['pres'], ds['temp'], cmap='viridis')
ax.invert_yaxis()
cbar = plt.colorbar(sc)
cbar.set_label('Temperature (°C)', fontsize=10)
ax.tick_params(axis="x", labelsize=8)
ax.tick_params(axis="y", labelsize=8)
ax.set_ylabel('Presure (dbar)', fontsize=10);
ax.set_xlabel('n_profiles', fontsize=10);

interpolate to standard levels

In [ ]:
std_lev = np.arange(0,max_depth)
ds_out = interpolate_standard_levels(ds, std_lev)

In [ ]:
# some format
#pres should be negative for the PCM
ds_out['PRES_INTERPOLATED'] = -np.abs(ds_out['PRES_INTERPOLATED'].values)
#axis attributtes for plotter class
ds_out.PRES_INTERPOLATED.attrs['axis'] = 'Z'
ds_out.lat.attrs['axis'] = 'Y'
ds_out.long.attrs['axis'] = 'X'
ds_out.dates.attrs['axis'] = 'T'

In [ ]:
ds_out

some plots

In [ ]:
ds_out['temp'].plot(x='n_profiles');

In [ ]:
proj=ccrs.PlateCarree()
subplot_kw = {'projection': proj}
fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(
            6, 6), dpi=120, facecolor='w', edgecolor='k', subplot_kw=subplot_kw)

sc = ax.scatter(ds['long'], ds['lat'], s=3, transform=proj)

land_feature = cfeature.NaturalEarthFeature(
            category='physical', name='land', scale='50m', facecolor=[0.9375, 0.9375, 0.859375])
ax.add_feature(land_feature, edgecolor='black')

## 2. Apply PCM

Create model

In [ ]:
z_dim = 'PRES_INTERPOLATED'
var_name_mdl = ['temp', 'sal']

# pcm feature
z = ds_out[z_dim]
pcm_features = {var_name_mdl[0]: z, var_name_mdl[1]: z}

m = pcm(K=K, features=pcm_features)
m

Fit model

In [ ]:
var_name_ds = ['temp', 'sal']
# Variable to be fitted {variable name in model: variable name in dataset}
features_in_ds = {var_name_mdl[0] : var_name_ds[0], var_name_mdl[1] : var_name_ds[1]}

m.fit_predict(ds_out, features=features_in_ds, dim=z_dim, inplace=True)
m

In [ ]:
m.predict_proba(ds_out, features=features_in_ds, dim=z_dim, inplace=True);

In [ ]:
ds_out = ds_out.pyxpcm.quantile(m, q=[0.05, 0.5, 0.95], of=var_name_ds[0], outname=var_name_ds[0] + '_Q', keep_attrs=True, inplace=True)
ds_out = ds_out.pyxpcm.quantile(m, q=[0.05, 0.5, 0.95], of=var_name_ds[1], outname=var_name_ds[1] + '_Q', keep_attrs=True, inplace=True)

In [ ]:
ds_out

## 3. BIC plot

User input

In [ ]:
corr_dist = 50 # correlation distance in km
time_steps = ['2018-01','2018-07']  # time steps to be used into account
Nrun = 10 # number of runs for each k
NK = 20 # max number of classes to explore

BIC calculation __(not std for the moment)__

In [ ]:
BIC = []

select_prof = np.arange(0,len(ds_out.n_profiles.values),6)
ds_run = ds_out.sel({'n_profiles': select_prof})
    
for i in range(1,NK):
    print(i)
    model = pcm(K=i, features=pcm_features)
    model.fit_predict(ds_run, features=features_in_ds, dim=z_dim)
    BIC.append(model.bic(ds_run, features=features_in_ds, dim=z_dim))
    
#BIC

BIC plot

In [ ]:
fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(
            6, 6), dpi=120, facecolor='w', edgecolor='k')

ax.plot(np.arange(1,NK), BIC)
plt.ylabel('BIC')
plt.xlabel('Number of classes')
plt.xticks(np.arange(NK) + 1)
plt.title('Bayesian information criteria (BIC)')

print('BIC min: ' + str(np.argmin(BIC) + 1))

## 4. Plots

In [ ]:
P = Plotter(ds_out, m, coords_dict= {'latitude': 'lat', 'longitude': 'long', 'time': 'dates'})
P.data_type = 'profiles'

vertical structure

In [ ]:
P.vertical_structure(q_variable = var_name_ds[0] + '_Q', sharey=True, xlabel='Temperature (°C)')

In [ ]:
P.vertical_structure(q_variable = var_name_ds[1] + '_Q', sharey=True, xlabel='Salinity (PSU)')

In [ ]:
P.vertical_structure_comp(q_variable = var_name_ds[0] + '_Q', plot_q= 'all', xlabel='Temperature (°C)')

In [ ]:
P.vertical_structure_comp(q_variable = var_name_ds[1] + '_Q', plot_q= 'all', xlabel='Salinity (PSU)')

spatial distribution

In [ ]:
P.spatial_distribution()

Classes pie chart

In [ ]:
P.pie_classes()

Temporal representation

Bar plot by month

In [ ]:
P.temporal_distribution(time_bins = 'month')

Bar plot by season

In [ ]:
P.temporal_distribution(time_bins = 'season')

## 4. Output

Create a .txt file with classes

In [ ]:
#create array with source, lat, long and PCM_labels
# change long values to 0-360 as used in OW
long_360 = np.mod(ds_out['long'].values, 360)
matrix_txt = np.stack((ds_out['source'].values, ds_out['lat'].values, long_360, ds_out['PCM_LABELS'].values), axis=1)
header = 'source lat long PCM_LABELS'

f= open("PCM_classes_NAtlac1_argo.txt","w+")
np.savetxt(f, matrix_txt, fmt=['%s', '%.3f', '%.3f', '%i'], header=header)
f.close() 